In [1]:
! pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.8/132.8 kB 1.7 MB/s eta 0:00:00


In [2]:
import pandas as pd
from sklearn.metrics import jaccard_score
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.cluster.hierarchy import linkage, dendrogram
import csv
import ast
import json

In [37]:
# file_path = "HateExplain_TEST_GPT_HUMAN_Rationales.csv"
file_path = "HateExplain_TRAIN_GPT_HUMAN_Rationales.csv"
dataset = pd.read_csv(file_path)
dataset

,id,annotators,rationales,post_tokens,PreProcessed_Text,ChatGPT Response,ChatGPT_Rationales,List_Of_Human_Rationales
0,23107796_gab,"{'label': [0, 2, 2], 'annotator_id': [203, 204...","[[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,...","['u', 'really', 'think', 'i', 'would', 'not', ...",u really think i would not have been raped by ...,"{\n ""text"": ""u really think i would not have ...","Rationales: NA; Derogatory Language: hindu, mu...","[['raped', 'neo', 'nazi', 'would', 'rape'], ['..."
1,9995600_gab,"{'label': [2, 2, 0], 'annotator_id': [27, 6, 4...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","['the', 'uk', 'has', 'threatened', 'to', 'retu...",the uk has threatened to return radioactive wa...,"{\n ""text"": ""the uk has threatened to return ...","Rationales: radioactive waste, paki migrants; ...","[['paki', 'migrants'], ['paki'], ['send', 'bac..."
2,1227920812235051008_twitter,"{'label': [2, 2, 2], 'annotator_id': [209, 203...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,...","['if', 'english', 'is', 'not', 'imposition', '...",if english is not imposition then hindi is als...,"{\n ""text"": ""if english is not imposition the...","Rationales: NA; Derogatory Language: chutiya, ...","[['chutiya', 'retards', 'stophindiimposition']..."
3,1204931715778543624_twitter,"{'label': [2, 2, 2], 'annotator_id': [235, 222...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1], [0, 0, 0, ...","['no', 'liberal', 'congratulated', 'hindu', 'r...",no liberal congratulated hindu refugees post c...,"{\n ""text"": ""no liberal congratulated hindu r...","Rationales: liberal, hindu refugees, cab; Dero...","[['hate', 'hindus'], ['hate', 'hindus']]"
4,1179102559241244672_twitter,"{'label': [2, 2, 2], 'annotator_id': [51, 25, ...","[[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,...","['he', 'said', 'bro', 'even', 'your', 'texts',...",he said bro even your texts sound redneck what...,"{\n ""text"": ""he said bro even your texts soun...",Rationales: NA; Derogatory Language: redneck; ...,"[['redneck'], ['redneck'], ['redneck']]"
...,...,...,...,...,...,...,...,...
15378,1125944647509917699_twitter,"{'label': [2, 2, 1], 'annotator_id': [217, 206...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0,...","['thanks', 'for', 'coming', 'to', 'my', 'ted',...",thanks for coming to my ted talk p.s. calling ...,"{\n ""text"": ""thanks for coming to my ted talk...",Rationales: NA; Derogatory Language: ching cho...,"[['ching', 'chong'], ['ching', 'chong']]"
15379,1191705189587341312_twitter,"{'label': [1, 1, 2], 'annotator_id': [209, 200...",[],"['<user>', '<user>', 'iran', 'has', 'the', '2'...",iran has the 2 n biggest community in middle e...,"{\n ""text"": ""iran has the 2 n biggest communi...",Rationales: NA; Derogatory Language: NA; Cuss ...,[]
15380,1178822728511606786_twitter,"{'label': [1, 1, 1], 'annotator_id': [127, 17,...",[],"['or', 'maybe', 'those', 'were', 'not', 'meant...",or maybe those were not meant to be hookups an...,"{\n ""text"": ""or maybe those were not meant to...",Rationales: NA; Derogatory Language: NA; Cuss ...,[]
15381,1179009825432358913_twitter,"{'label': [1, 1, 1], 'annotator_id': [49, 18, ...",[],"['good', 'morning', 'ados', 'black', 'women', ...",good morning ados black women only,"{\n ""text"": ""good morning ados black women on...",Rationales: ados black women; Derogatory Langu...,[]


# Union Function

In [38]:
def get_union(list_of_lists):
    if not list_of_lists or not all(isinstance(lst, list) for lst in list_of_lists):
        return []

    union_set = set(list_of_lists[0]).union(*map(set, list_of_lists[1:]))
    union_list = list(union_set)

    return union_list

#Calculating Overlap, Jaccard Similarity

In [39]:
import csv
import ast
import pandas as pd

updated_file_path = 'HateExplain_TRAIN_GPT_HUMAN_Rationales_Overlap_Coefficient.csv'

with open(updated_file_path, mode='w', newline='') as updated_file:
    csv_writer = csv.writer(updated_file)

    csv_writer.writerow(dataset.columns.tolist() + ['Human Rationales List'] + ['LLM Rationales List'])

    for index, row in dataset.iterrows():
        human_rationales = ast.literal_eval(row['List_Of_Human_Rationales'])
        human_rationaes_set = get_union(human_rationales)
        # print("human_rationaes_set: ", human_rationaes_set)
        try:
            # llm_rationales = ast.literal_eval(row['ChatGPT Response'])
            llm_rationales = json.loads((row['ChatGPT Response']))
            # print(type(llm_rationales.get("rationales")))
            rationales = list(llm_rationales.get("rationales"))
            rationale_list = []
            for phrase in rationales:
                rationale_list.extend(phrase.split())
            # print(type(rationale_list))
            derogatory_language_list = list(llm_rationales.get("derogatory_language"))
            cuss_words_list = list(llm_rationales.get("cuss_words"))

            rationale_list.extend(derogatory_language_list)
            rationale_list.extend(cuss_words_list)
            llm_rationales_list = list(rationale_list)
            llm_rationales_set = list(set(llm_rationales_list))
            # print("llm_rationales_set: ", llm_rationales_set)

        except json.JSONDecodeError:
            print("Error: llm_rationale is not in JSON format")
            llm_rationales_set = set()

        row_values = row.tolist()
        row_values.append(human_rationaes_set)
        row_values.append(llm_rationales_set)

        csv_writer.writerow(row_values)

updated_data = pd.read_csv(updated_file_path)


Error: llm_rationale is not in JSON format
Error: llm_rationale is not in JSON format
Error: llm_rationale is not in JSON format
Error: llm_rationale is not in JSON format
Error: llm_rationale is not in JSON format
Error: llm_rationale is not in JSON format
Error: llm_rationale is not in JSON format
Error: llm_rationale is not in JSON format
Error: llm_rationale is not in JSON format
Error: llm_rationale is not in JSON format
Error: llm_rationale is not in JSON format
Error: llm_rationale is not in JSON format
Error: llm_rationale is not in JSON format
Error: llm_rationale is not in JSON format
Error: llm_rationale is not in JSON format
Error: llm_rationale is not in JSON format
Error: llm_rationale is not in JSON format
Error: llm_rationale is not in JSON format
Error: llm_rationale is not in JSON format
Error: llm_rationale is not in JSON format
Error: llm_rationale is not in JSON format
Error: llm_rationale is not in JSON format
Error: llm_rationale is not in JSON format
Error: llm_

# Removing Stopwords from both Human and LLM Rationales Lists

In [40]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
import string

In [41]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [42]:
punct = set(string.punctuation)
stop_words = set(stopwords.words('english'))
unwanted = punct | stop_words

def remove_unwanted(words):
    words = ast.literal_eval(words)
    return [word for word in words if word.lower() not in unwanted]

updated_data['Human Rationales List Without StopWords'] = updated_data['Human Rationales List'].apply(remove_unwanted)
updated_data['LLM Rationales List Without StopWords'] = updated_data['LLM Rationales List'].apply(remove_unwanted)

In [43]:
updated_data

,id,annotators,rationales,post_tokens,PreProcessed_Text,ChatGPT Response,ChatGPT_Rationales,List_Of_Human_Rationales,Human Rationales List,LLM Rationales List,Human Rationales List Without StopWords,LLM Rationales List Without StopWords
0,23107796_gab,"{'label': [0, 2, 2], 'annotator_id': [203, 204...","[[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,...","['u', 'really', 'think', 'i', 'would', 'not', ...",u really think i would not have been raped by ...,"{\n ""text"": ""u really think i would not have ...","Rationales: NA; Derogatory Language: hindu, mu...","[['raped', 'neo', 'nazi', 'would', 'rape'], ['...","['neo', 'raped', 'hindu', 'nazi', 'rape', 'mus...","['muslim', 'neo nazi', 'hindu']","[neo, raped, hindu, nazi, rape, muslim, feral,...","[muslim, neo nazi, hindu]"
1,9995600_gab,"{'label': [2, 2, 0], 'annotator_id': [27, 6, 4...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","['the', 'uk', 'has', 'threatened', 'to', 'retu...",the uk has threatened to return radioactive wa...,"{\n ""text"": ""the uk has threatened to return ...","Rationales: radioactive waste, paki migrants; ...","[['paki', 'migrants'], ['paki'], ['send', 'bac...","['all', 'the', 'paki', 'instead', 'send', 'bac...","['migrants', 'waste', 'radioactive', 'paki']","[paki, instead, send, back, migrants]","[migrants, waste, radioactive, paki]"
2,1227920812235051008_twitter,"{'label': [2, 2, 2], 'annotator_id': [209, 203...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,...","['if', 'english', 'is', 'not', 'imposition', '...",if english is not imposition then hindi is als...,"{\n ""text"": ""if english is not imposition the...","Rationales: NA; Derogatory Language: chutiya, ...","[['chutiya', 'retards', 'stophindiimposition']...","['stophindiimposition', 'chutiya', 'retards']","['chutiya', 'retards']","[stophindiimposition, chutiya, retards]","[chutiya, retards]"
3,1204931715778543624_twitter,"{'label': [2, 2, 2], 'annotator_id': [235, 222...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1], [0, 0, 0, ...","['no', 'liberal', 'congratulated', 'hindu', 'r...",no liberal congratulated hindu refugees post c...,"{\n ""text"": ""no liberal congratulated hindu r...","Rationales: liberal, hindu refugees, cab; Dero...","[['hate', 'hindus'], ['hate', 'hindus']]","['hate', 'hindus']","['hindus', 'hindu', 'cab', 'refugees', 'liberal']","[hate, hindus]","[hindus, hindu, cab, refugees, liberal]"
4,1179102559241244672_twitter,"{'label': [2, 2, 2], 'annotator_id': [51, 25, ...","[[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,...","['he', 'said', 'bro', 'even', 'your', 'texts',...",he said bro even your texts sound redneck what...,"{\n ""text"": ""he said bro even your texts soun...",Rationales: NA; Derogatory Language: redneck; ...,"[['redneck'], ['redneck'], ['redneck']]",['redneck'],"['redneck', 'fuck']",[redneck],"[redneck, fuck]"
...,...,...,...,...,...,...,...,...,...,...,...,...
15378,1125944647509917699_twitter,"{'label': [2, 2, 1], 'annotator_id': [217, 206...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0,...","['thanks', 'for', 'coming', 'to', 'my', 'ted',...",thanks for coming to my ted talk p.s. calling ...,"{\n ""text"": ""thanks for coming to my ted talk...",Rationales: NA; Derogatory Language: ching cho...,"[['ching', 'chong'], ['ching', 'chong']]","['chong', 'ching']",['ching chong'],"[chong, ching]",[ching chong]
15379,1191705189587341312_twitter,"{'label': [1, 1, 2], 'annotator_id': [209, 200...",[],"['<user>', '<user>', 'iran', 'has', 'the', '2'...",iran has the 2 n biggest community in middle e...,"{\n ""text"": ""iran has the 2 n biggest communi...",Rationales: NA; Derogatory Language: NA; Cuss ...,[],[],[],[],[]
15380,1178822728511606786_twitter,"{'label': [1, 1, 1], 'annotator_id': [127, 17,...",[],"['or', 'maybe', 'those', 'were', 'not', 'meant...",or maybe those were not meant to be hookups an...,"{\n ""text"": ""or maybe those were not meant to...",Rationales: NA; Derogatory Language: NA; Cuss ...,[],[],[],[],[]
15381,1179009825432358913_twitter,"{'label':

In [44]:
def overlap_coeff(list1, list2):
    intersection = len(set(list1).intersection(set(list2)))
    min_len = min(len(list1), len(list2))
    if min_len == 0:        #??
        min_len = 1
    return intersection / min_len

def jaccard_similarity(list1, list2):
    intersection = len(set(list1).intersection(set(list2)))
    union = len(set(list1).union(set(list2)))
    if union == 0:        #??
        union = 1
    return intersection / union

In [45]:
jaccards = []
overlaps = []
for index, row in updated_data.iterrows():
    # list1 = ast.literal_eval(row['Human Rationales List'])
    list1 = row['Human Rationales List Without StopWords']
    list2 = row['LLM Rationales List Without StopWords']

    jaccard = jaccard_similarity(list1, list2)
    jaccards.append(jaccard)

    overlap = overlap_coeff(list1, list2)
    overlaps.append(overlap)

updated_data['Overlap_New'] = overlaps
updated_data['Jaccard_New'] = jaccards


#Overlap and Jaccard Similarity Mean

In [46]:
print(updated_data['Overlap_New'].mean())
print(updated_data['Jaccard_New'].mean())

0.4657700389568548
0.26216063090612435


#Cosine Similarity Between 'Human Rationales List Without StopWords' and 'LLM Rationales List Without StopWords'

In [47]:
import spacy
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [48]:
! python -m spacy download en_core_web_md

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 10.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [49]:
nlp = spacy.load("en_core_web_md")

In [50]:
# Function to get sentence embeddings
def get_sentence_embedding(sentence):
    return np.mean([word.vector for word in nlp(sentence) if word.has_vector], axis=0)

similarities = []

for index, row in updated_data.iterrows():
    col1_embedding = get_sentence_embedding(' '.join(row['Human Rationales List Without StopWords']))
    col2_embedding = get_sentence_embedding(' '.join(row['LLM Rationales List Without StopWords']))

    if np.isnan(np.sum(col1_embedding)) or np.isnan(np.sum(col2_embedding)):
        similarities.append(np.nan)
    else:
        similarity = cosine_similarity([col1_embedding], [col2_embedding])[0][0]
        similarities.append(similarity)

updated_data['Cosine_Similarity'] = similarities

Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret

In [51]:
updated_data

,id,annotators,rationales,post_tokens,PreProcessed_Text,ChatGPT Response,ChatGPT_Rationales,List_Of_Human_Rationales,Human Rationales List,LLM Rationales List,Human Rationales List Without StopWords,LLM Rationales List Without StopWords,Overlap_New,Jaccard_New,Cosine_Similarity
0,23107796_gab,"{'label': [0, 2, 2], 'annotator_id': [203, 204...","[[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,...","['u', 'really', 'think', 'i', 'would', 'not', ...",u really think i would not have been raped by ...,"{\n ""text"": ""u really think i would not have ...","Rationales: NA; Derogatory Language: hindu, mu...","[['raped', 'neo', 'nazi', 'would', 'rape'], ['...","['neo', 'raped', 'hindu', 'nazi', 'rape', 'mus...","['muslim', 'neo nazi', 'hindu']","[neo, raped, hindu, nazi, rape, muslim, feral,...","[muslim, neo nazi, hindu]",0.666667,0.222222,0.851543
1,9995600_gab,"{'label': [2, 2, 0], 'annotator_id': [27, 6, 4...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","['the', 'uk', 'has', 'threatened', 'to', 'retu...",the uk has threatened to return radioactive wa...,"{\n ""text"": ""the uk has threatened to return ...","Rationales: radioactive waste, paki migrants; ...","[['paki', 'migrants'], ['paki'], ['send', 'bac...","['all', 'the', 'paki', 'instead', 'send', 'bac...","['migrants', 'waste', 'radioactive', 'paki']","[paki, instead, send, back, migrants]","[migrants, waste, radioactive, paki]",0.500000,0.285714,0.730355
2,1227920812235051008_twitter,"{'label': [2, 2, 2], 'annotator_id': [209, 203...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,...","['if', 'english', 'is', 'not', 'imposition', '...",if english is not imposition then hindi is als...,"{\n ""text"": ""if english is not imposition the...","Rationales: NA; Derogatory Language: chutiya, ...","[['chutiya', 'retards', 'stophindiimposition']...","['stophindiimposition', 'chutiya', 'retards']","['chutiya', 'retards']","[stophindiimposition, chutiya, retards]","[chutiya, retards]",1.000000,0.666667,1.000000
3,1204931715778543624_twitter,"{'label': [2, 2, 2], 'annotator_id': [235, 222...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1], [0, 0, 0, ...","['no', 'liberal', 'congratulated', 'hindu', 'r...",no liberal congratulated hindu refugees post c...,"{\n ""text"": ""no liberal congratulated hindu r...","Rationales: liberal, hindu refugees, cab; Dero...","[['hate', 'hindus'], ['hate', 'hindus']]","['hate', 'hindus']","['hindus', 'hindu', 'cab', 'refugees', 'liberal']","[hate, hindus]","[hindus, hindu, cab, refugees, liberal]",0.500000,0.166667,0.621271
4,1179102559241244672_twitter,"{'label': [2, 2, 2], 'annotator_id': [51, 25, ...","[[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,...","['he', 'said', 'bro', 'even', 'your', 'texts',...",he said bro even your texts sound redneck what...,"{\n ""text"": ""he said bro even your texts soun...",Rationales: NA; Derogatory Language: redneck; ...,"[['redneck'], ['redneck'], ['redneck']]",['redneck'],"['redneck', 'fuck']",[redneck],"[redneck, fuck]",1.000000,0.500000,0.807234
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15378,1125944647509917699_twitter,"{'label': [2, 2, 1], 'annotator_id': [217, 206...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0,...","['thanks', 'for', 'coming', 'to', 'my', 'ted',...",thanks for coming to my ted talk p.s. calling ...,"{\n ""text"": ""thanks for coming to my ted talk...",Rationales: NA; Derogatory Language: ching cho...,"[['ching', 'chong'], ['ching', 'chong']]","['chong', 'ching']",['ching chong'],"[chong, ching]",[ching chong],0.000000,0.000000,1.000000
15379,1191705189587341312_twitter,"{'label': [1, 1, 2], 'annotator_id': [209, 200...",[],"['<user>', '<user>', 'iran', 'has', 'the', '2'...",iran has the 2 n biggest community in middle e...,"{\n ""text"": ""iran has the 2 n biggest communi...",Rationales: NA; Derogatory Language: NA; Cuss ...,[],[],[],[],[],0.000000,0.000000,NaN
15380,1178822728511606786_twitter,"{'label': [1, 1, 1], 'annotator_id': [127, 17,...",[],"['or', 'maybe', 'those', '

In [52]:
updated_data['Cosine_Similarity'].mean()

0.7425388670864372

In [54]:
updated_data.to_excel('HateExplain_TRAIN_GPT_HUMAN_Rationales_Similarity_Coefficients.xlsx')
updated_data.to_csv('HateExplain_TRAIN_GPT_HUMAN_Rationales_Similarity_Coefficients.csv')

In [55]:
from google.colab import files
files.download('HateExplain_TRAIN_GPT_HUMAN_Rationales_Similarity_Coefficients.xlsx')
files.download('HateExplain_TRAIN_GPT_HUMAN_Rationales_Similarity_Coefficients.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#Visualization for Similarity

In [53]:
from IPython.display import HTML
import ast
import pandas as pd

# User-specified index
user_index = 3

text = updated_data['PreProcessed_Text'][user_index]
human_rationale = updated_data['Human Rationales List Without StopWords'][user_index]
llm_rationale = updated_data['LLM Rationales List Without StopWords'][user_index]

# Set colors
human_set = set(human_rationale)
llm_set = set(llm_rationale)
common = human_set.intersection(llm_set)
human_only = human_set - common
llm_only = llm_set - common

# Format the text
formatted_text = ""
for word in text.split():
    if word in human_only:
        formatted_text += f"<span style='color: red'>{word}</span> "
    elif word in common:
        formatted_text += f"<span style='color: purple'>{word}</span> "
    elif word in llm_only:
        formatted_text += f"<span style='color: blue'>{word}</span> "
    else:
        formatted_text += f"{word} "

# Display the formatted text
html_text = f"<span>{formatted_text}</span>"
display(HTML(html_text))